# Perkenalan

# GC3



*   Nama : Iqbal Awis Nurdiansyah
*   Batch : RMT-19

**Latar belakang** : saya adalah seorang data analis di The Look yang merupakan salah satu platform e-commerce terbesar di planet Mars. Saya diminta untuk membuat laporan evaluasi aktivitas penjualan di platform tersebut



**Objectiv**: Memberikan Insight dan masukan kepada CEO terkait data penjualan perusahaan

**Problem Statement** : dalam menentukan problem statment kali ini menggunakan Metode SMART



*   `Spesific`: Melihat dan mengetahui data penjualan yang terjadi di The Look selama tahun 2022 dari bulan Januari sampai September 
*   `Measurable` : Banyaknya jumlah transaksi dan total pendapatan yang terjadi selama tahun 2022 dari bulan Januari sampai September dan hubungan/korelasi data dengan customer
*   `Achievable` : Pengolahan data menggunakan metode data wrangling atau preprocessing untuk memberikan insight 
*   `Relevant` : memeberikan insight yang relevan dan bisa diterima dengan masuk akal
*   `Time-Bond` : data yang digunakan yakni tahun 2022 antara bulan Januari - September



# Import Library

In [1]:
#import library
import pandas as pd
from google.cloud import bigquery

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

project_id = "sqlbatch19" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


Notebook berhasil terauntentikasi dengan google bigquery

## Soal 1 Berapa jumlah transaksi yang berstatus Complete tiap bulan selama Q1 sampai Q3 di tahun 2022? Insight apa yang bisa kamu berikan?

In [3]:
#meload jumlah transaksi berstatus complete
df_jumlah_transaksi= client.query('''
select extract(year FROM created_at) as year,
       extract(month from created_at) as  month,
       extract(Quarter from created_at) as quartile_month,
        count (order_id ) as jumlah_transaksi
from `bigquery-public-data.thelook_ecommerce.orders`
where status='Complete' and extract (year from created_at)=2022 and extract (Quarter from created_at)between 1 and 3
group by year,month,quartile_month
order by month asc;

''').to_dataframe()

In [4]:
df_jumlah_transaksi

,year,month,quartile_month,jumlah_transaksi
0,2022,1,1,717
1,2022,2,1,704
2,2022,3,1,768
3,2022,4,2,865
4,2022,5,2,908
5,2022,6,2,976
6,2022,7,3,1070
7,2022,8,3,1144
8,2022,9,3,1185




*   jumlah transaksi tiap bulan pada quartil ke-1 tiap bulannya adalah = 731, 639, 798
*   jumlah transaksi tiap bulan pada quartil ke-2 tiap bulannya adalah = 814, 896, 949
*   jumlah transaksi tiap bulan pada quartil ke-3 tiap bulannya adalah = 1058, 1143, 1139

**Jumlah transaksi mengalami kenaikan tertinggi pada quartil ke-3**




## Soal 2 Berikan informasi total penjualan (dalam USD) tiap bulan selama Q1 sampai Q3 di tahun 2022! (Hanya yang transaksi berstatus Complete. Apa informasi yang bisa kamu sampaikan?

In [5]:
#menampilkan total data penjualan
df_total_penjualan= client.query('''
select extract(year FROM created_at) as year,
       extract(month from created_at) as  month,
        extract(Quarter from created_at) as quartile_month,
        sum (sale_price) as price
from `bigquery-public-data.thelook_ecommerce.order_items`
where status='Complete' and extract (year from created_at)=2022 and extract (Quarter from created_at)between 1 and 3
group by year,month,quartile_month
order by month asc;

''').to_dataframe()
df_total_penjualan

,year,month,quartile_month,price
0,2022,1,1,63376.380109
1,2022,2,1,61518.120096
2,2022,3,1,68069.050040
3,2022,4,2,73546.890092
4,2022,5,2,79337.690119
5,2022,6,2,87005.690090
6,2022,7,3,91475.250053
7,2022,8,3,101345.860173
8,2022,9,3,104123.500200


*   total penjualan tiap bulan pada quartil ke-1 tiap bulannya adalah = 63376,61518 , 68069
*   total penjualan tiap bulan pada quartil ke-2 tiap bulannya adalah = 73546, 79337, 87005
*   total penjualan tiap bulan pada quartil ke-3 tiap bulannya adalah = 91475, 101345, 104123

**total penjualan mengalami kenaikan tertinggi pada quartil ke-3
dan total penjualan terendah berada pada bulan ke-2 pada quartil ke-1**


## Soal 3 Berapa user yang melakukan transaksi berstatus Complete di tiap bulan dari Q1 sampai Q3 2022? Apa kesimpulanmu?

In [6]:
#total data pembeli
df_total_pembeli= client.query('''
select extract(year FROM created_at) as year,
  extract (month from created_at) as month,
  extract (Quarter from created_at ) as quartile_month,
  count (distinct user_id) as jumlah_user 
from `bigquery-public-data.thelook_ecommerce.order_items`
where status='Complete' and extract (year from created_at)=2022 and extract (Quarter from created_at)between 1 and 3
group by year,month,quartile_month
order by month asc;

''').to_dataframe()
df_total_pembeli

,year,month,quartile_month,jumlah_user
0,2022,1,1,737
1,2022,2,1,711
2,2022,3,1,781
3,2022,4,2,878
4,2022,5,2,915
5,2022,6,2,991
6,2022,7,3,1088
7,2022,8,3,1156
8,2022,9,3,1198




*   Jumlah Pembeli pada quartal 1 : 737,711,781 = 2.229 user
*   Jumlah Pembeli pada quartal 2 : 878,915,911 = 2.704 user
*   Jumlah Pembeli pada quartal 3 : 1088,1156,1198 = 3.442 user

**Jumlah pembeli meningkat seiring bertambah nya kuartal**



## soal 4 Kategori produk apa saja yang paling banyak dibeli (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [7]:
df_cat_product= client.query('''
SELECT year, month, category as most_bought, transaction #query 1
FROM (
      SELECT category, year, month, transaction,#query 2
      ROW_NUMBER() OVER (PARTITION BY year, month ORDER BY transaction DESC) AS rn
      FROM (
            SELECT products.category,#query 3
            EXTRACT(YEAR FROM created_at) AS year,
            EXTRACT(MONTH FROM created_at) AS month,
            COUNT(category) AS transaction
            FROM bigquery-public-data.thelook_ecommerce.order_items AS order_items
            JOIN bigquery-public-data.thelook_ecommerce.products AS products
            ON order_items.product_id = products.id
            WHERE status = 'Complete' AND EXTRACT(YEAR FROM created_at) = 2022 
            AND EXTRACT(QUARTER FROM created_at) BETWEEN 1 AND 3
            GROUP BY category, year, month))
WHERE rn = 1
ORDER BY month ASC;
''').to_dataframe()
df_cat_product

,year,month,most_bought,transaction
0,2022,1,Shorts,79
1,2022,2,Jeans,79
2,2022,3,Intimates,94
3,2022,4,Fashion Hoodies & Sweatshirts,113
4,2022,5,Intimates,96
5,2022,6,Intimates,119
6,2022,7,Tops & Tees,118
7,2022,8,Jeans,131
8,2022,9,Intimates,129




*   Pada kuartal pertama yaitu bulan januari-maret jenis produk `Intimates` menjadi produk paling banyak dibeli
*   Pada kuartal kedua yaitu bulan april-juni jenis produk `Intimates` menjadi produk paling banyak dibeli
*   Pada kuartal ketiga yaitu bulan juli-september jenis produk `Jeans` menjadi produk paling banyak dibeli





## soal  5 Kategori produk apa saja yang paling banyak mendapatkan pendapatan (staus transaksi: Complete) di tiap bulannya selama Q1 sampai Q3 tahun 2022? Berikan insight!

In [8]:
df_cat_product_most= client.query('''
SELECT 
  month,
  category,
  total_revenue
FROM(
  SELECT 
    category, 
    month, 
    total_revenue, 
  RANK() OVER (PARTITION BY month ORDER BY total_revenue DESC) AS rank 
    FROM(
      SELECT 
        category, 
        ROUND(SUM(sale_price),2) AS total_revenue, 
        EXTRACT(MONTH FROM items.delivered_at) AS month 
      FROM bigquery-public-data.thelook_ecommerce.order_items AS items
      JOIN bigquery-public-data.thelook_ecommerce.products product
        ON (items.product_id = product.id)
      WHERE items.status = 'Complete'
        AND EXTRACT(YEAR FROM items.delivered_at) = 2022
        AND EXTRACT(QUARTER FROM items.delivered_at) BETWEEN 1 AND 3
      GROUP BY category, month))
WHERE rank = 1
ORDER BY month, total_revenue DESC
''').to_dataframe()
df_cat_product_most

,month,category,total_revenue
0,1,Outerwear & Coats,7154.92
1,2,Outerwear & Coats,8598.97
2,3,Outerwear & Coats,9431.65
3,4,Outerwear & Coats,10910.89
4,5,Jeans,9125.05
5,6,Jeans,10719.55
6,7,Outerwear & Coats,10982.93
7,8,Jeans,12884.20
8,9,Jeans,13056.27


* kuartal 1 : 25.183
* kuartal 2 : 30.754
* kuartal 3 : 36.922
dari keseluruhan revenue yang didapat dari kuartal 1-3 jenis barang dengan katagori `Jeans` berhasil meraup keuntungan sebesar **13.056** pada kuartal ke 3,lebih tepatnya pada bulan ke-9, Jumlah total revenue sebesar : **92.859**

## Soal 6 Dibandingkan dengan jumlah transaksi dan total penjualan, mana yang paling berkaitan dengan jumlah user yang melakukan transaksi? Apa analisis yang dapat kamu jelaskan?

In [9]:
#melihat korelasi
df_corr= client.query('''SELECT 
ROUND(CORR(user, sales_complete), 5) user_vs_transactions, 
ROUND(CORR(user, total_sales), 5) user_vs_sales
FROM( 
  SELECT 
  EXTRACT(year from created_at) as year,
  EXTRACT(month from created_at) as month,
  COUNT(DISTINCT order_id) as sales_complete, --jumlah transaksi
  SUM(sale_price) as total_sales, --jumlah penjualan
  COUNT(DISTINCT user_id) as user --jumlah user
  FROM bigquery-public-data.thelook_ecommerce.order_items items
  WHERE status = 'Complete' 
    AND EXTRACT(YEAR FROM items.delivered_at) = 2022
    AND EXTRACT(QUARTER FROM items.delivered_at) BETWEEN 1 AND 3
  GROUP BY year,month
  ORDER BY year,month ASC)
  ''').to_dataframe()
df_corr

,user_vs_transactions,user_vs_sales
0,0.99998,0.999


korelasi antara user dan transaksi sangat berhubungan karena semakin banyak user akan semakin banyak pula jumlah transaksi, jika semakin banyak transaksi pendapatanpun akan semakin meningkat.

## Pertanyaan

### 1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

Problem Statment dapat dicapai dengan melihat data penjualan dari kuartal 1-3, penjualan selalu meningkat seiring masuk nya kuartal baru, baik dalam segi penjualan dan pendapatan.

### 2. Berdasarkan hasil analisis yang sudah kamu lakukan dari 6 penjabaran di atas, jika CEO perusahaanmu ingin menargetkan pendapatan di awal kuartal 4 harus mencapai $250000, apakah masuk akal?

* kuartal 1 : 25.183
* kuartal 2 : 30.754
* kuartal 3 : 36.922

kenaikan revenue dari kuartal 1 ke 2 adalah : 5.571, kuartal 2 ke 3 adalah : 6.168. Memang disetiap kuartal mengalami kenaikan revenue, tapi untuk mencapai revenue 250.000, dirasa terlalu berlebihan karena selisih antar revenue bahkan tidak sampai 3.000.


### 3. CEO kamu menargetkan di kuartal 4 ada investor yang dapat menyuntikan dana ke perusahaanmu, dimana investor akan melihat GMV selama 3 kuartal terakhir serta prospek kedepan minimal di kuartal 4 akan seperti apa. Berikan informasi kepada CEO mu berdasarkan trend transaksi, jumlah user yang bertransaksi, dan GMV 3 kuartal terakhir, apakah perusahaanmu layak atau tidak mendapatkan investor baru!

Saya rasa perusahaan layak mendapatkan suntikan dana karena data menunjukan semakin dekat dengan akhir tahun tren membeli barang sangat meningkat, diprediksi akan ada peningkatan yang cukup signifikan pada kuartal ke-4, karena menjelang event natal dan tahun baru.

## Kesimpulan

Kesimpulan akhir yang didapatkan adalah semakin bertambahnya jumlah kuartal jumlah transaksi diprediksi akan terus meningkat apalagi menjelang hari raya natal dan tahun baru. Sekiranya perusahan lebih bersiap untuk menangani jumlah pemesanan yang cukup banyak  bahkan jika memang diperlukan, adanya investor akan sangat baik untuk mendapatkan modal untuk  menambah jumlah barang yang siap untuk dijual.